# Rapport de projet final: Débordement d'égouts
## Équipe 9 - A19
#### _Noboru Yoshida - Mehdi Chaid - Mathieu Giroux-Huppé - Maxime Gosselin_
<br>
20 Décembre 2019

## Table des matières

+ [1. Introduction](#1.)
    + [1.1. Objectifs](#1.1.)  
    + [1.2. Méthode de travail](#1.2.)  
<br>
+ [2. Analyse exploratoire](#2.)
    + [2.1. Motivation](#2.1.)  
    + [2.2. Importation des données](#2.2.)  
    + [2.3. Fonctions utilisées](#2.3.)
    + [2.4. Analyse des données d'ouvrages](#2.4.)  
        + [2.4.1. Données discrètes](#2.4.1.)
        + [2.4.2. Données de Trop-Plein](#2.4.2.)
        + [2.4.3. Données d'Emissaire](#2.4.3.)
    + [2.5. Analyse des données de précipitations](#2.5.)  
        + [2.5.1. Données de précipitations](#2.5.1.)
        + [2.5.2. Somme des précipitations journalières](#2.5.2.)
        + [2.5.3. Taux horaire maximal de précipitations journalières](#2.5.3.)
        + [2.5.4. Taux sur trois heures maximal des précipitations journalières](#2.5.4.)
    + [2.6. Analyse des données de surverses](#2.6.)
        + [2.6.1. Chargement des surverses pour analyse](#2.6.1.)
        + [2.6.2. Lien entre ouvrages et surverses](#2.6.2.)
        + [2.6.3. Lien entre précipitations de surverses](#2.6.3.)
    + [2.7. Isolation des ouvrages d'intérêts](#2.7.)
        + [2.7.1. Motivation](#2.7.1.)
        + [2.7.2. Chargement des données traitées](#2.7.2.)
        + [2.7.3. Station la plus proche](#2.7.3.)
        + [2.7.4. Deuxième station la plus proche](#2.7.4.)
    + [2.8. Retour sur l'analyse](#2.8.)
<br>


+ [3. Traitement des données](#3.)
    + [3.1. Motivation](#3.1.)  
    + [3.2. Processus de traitement](#3.2.)
    + [3.3. Fonctions utilisées](#3.3.)
    + [3.4. Considérations et étapes retenues](#3.4.)
    + [3.5. Isolation des ouvrages](#3.6.)
    + [3.6. Rafinements isolés](#3.7.)
<br>


+ [4. Sélection de modèles](#4.)
    + [4.1. Motivation](#4.1)
    + [4.2. Choix des modèles](#4.2.)
    + [4.3. Arbres de décision et forêt aléatoire](#4.3.)  
    + [4.4. Régression logistique](#4.4.)  
    + [4.5. Machine à vecteurs de support](#4.5.)  
    + [4.6. Classification bayésienne naive](#4.6.)  
    + [4.7. Ensemble de modèles](#4.7.)  
<br>
+ [5. Retour et conclusion](#5.)
    + [5.1. Retour sur les résultats](#5.1.)  
    + [5.2. Améliorations possibles](#5.2.)
    + [5.3. Difficultés rencontrées](#5.3.)
        + [5.3.1. Interpretation des données](#5.3.1.)
    + [5.4. Conclusion](#5.5.)  
<br>

+ [6. Références](#refs)

<a id="1."></a>

## 1. Introduction

<a id="1.1."></a>
### 1.1. Objectifs

Ce rapport vise à fournir une prédiction sur les surverses dans plusieurs ouvrages sur l'île de Montréal. Il en existe 170 répartis sur tout le bord de l'île. Dans ce travail, nous nous attarderons sur la prédiction de seulement 5 d'entre-eux, soit les suivants :
- 3260-01D dans Rivière-des-Prairies
- 3350-07D dans Ahunstic
- 4240-01D dans Pointe-aux-Trembles
- 4350-01D dans le Vieux-Montréal
- 4380-01D dans Verdun

Nous avons à notre disposition 3 jeux de données qui nous aiderons à trouver une relation entre la quantité de pluie tombé et les surverses de certain ouvrages. Nous devrons alors entraîner un modèle sur les données de 2013 à 2018, puis prédire sur certaines dates de 2019. Il n'y pas de restrictions au niveau des techniques à utilisés pour le type de modèle à utiliser. Nous avons exploré quelques techniques différentes que nous allons détaillé plus en détail dans la [section 5](#model-selection).

<a id="1.2."></a>
### 1.2. Déroulement de travail

Le projet s'est déroulé de la sorte: 

 1. Tout d'abord, nous avons effectué une analyse superficielle des données afin de comprendre leur sens et de pouvoir commencer la tâche.

 2. Puis, nous avons procédé à une exploration initiale des modèles selon les données que nous avons et en pesant les avantages et inconvénients de chaque modèle afin de déterminer les plus appropriés.

 3. Par la suite, nous avons effectué un traitement initiale des données afin de les intégrer aux modèles.

 4. À cette étape, nous avons appliqué les multiples modèles choisis à nos données traitées afin de les analyser et d'isoler les plus performants.

 5. Nous avons ensuite procédé à une analyse plus poussée afin de trouver des manière d'améliorer nos données.

 6. Nous avons alors traité plus en profondeur les données pour les adapter à la situation et rendre nos modèles plus robustes.

 7. Finalement, nous avons rafiné les modèles selon ces nouvelles données, menant à nos prédictions finales.

Nous aborderons dans ce rapport chacune de ces parties, séparées en 3 groupes: l'analyse exploratoire, le traitement de données et la sélection de modèle. 

<a id="2."></a>
## 2. Analyse exploratoire

<a id="2.1."></a>
### 2.1. Motivation

L'analyse exploratoire est une très importante pour tout exercice en sciences de données. Elle permet de déterminer, entres autres:

- Quelles variables vont jouer un rôle important pour la prédiction.
- À quel point les variables sont corrélées entre elles.
- Si une régularisation est nécessaire pour certaines variables.
- Comment améliorer le temps d'entrainement et les prédictions.

Pour ce faire, nous allons analyser chacune des variables explicatives en utilisant des graphes et des statistiques afin d'appuyer nos résultats et de guider notre prise de décision.

Cette section du rapport, qui est assez longue, peut être ignorée si le lecteur est déjà familier avec les jeux de données utilisés. Elle permet néanmoins de mieux comprendre comment nous avons choisi les variables d'intérêts et comment nous sommes arrivés aux conclusions que nous avons tiré lors de la préparation de modèles.

<a id="2.2."></a>
### 2.2. Importation des données

Installons d'abord les libraires utilisées pour l'analyse exploratoire.

In [ ]:
using Pkg;
Pkg.add("CSV");
Pkg.add("Random");
Pkg.add("DataStructures");
Pkg.add("BenchmarkTools");
Pkg.add("DataFrames");
Pkg.add("Statistics");
Pkg.add("Dates");
Pkg.add("Gadfly");
Pkg.add("IterTools");

Importons maintenant ces librairies dans le rapport.

In [1]:
using CSV, DataFrames, DataStructures, BenchmarkTools, Statistics, Dates, Gadfly, Random, IterTools;

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/chaime/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


Configuration de l'outil de visualisation.

In [2]:
set_default_plot_size(25cm, 13cm)

<a id="2.3."></a>
### 2.3. Fonctions utilisées [TODO: COMPLETER]

Nous définissons ici les fonctions utilisées pour l'analyse exploratoire de données. Ces fonctions sont implémentées dans le fichier suivant afin de ne pas polluer le rapport.

In [3]:
include("utils/precipitation.jl");

<a id="2.3.1."></a>
#### 2.3.1. random_first

Cette function est un remplacement à la fonction first qui nous permet d'obtenir un échantillon aléatoire de données.

In [4]:
function random_first(df::DataFrame, nb::Int64, fts::Array=[])
    if length(fts) == 0
        first(df[shuffle(1:size(df, 1)),:], nb)
    else
        first(df[shuffle(1:size(df, 1)),fts], nb)
    end
end

random_first (generic function with 2 methods)

<a id="2.3.2."></a>
#### 2.3.2. maximum_3

#### TODO: Describe

<a id="2.4."></a>
### 2.4. Analyse des données d'ouvrages

Commençons par importer le fichier de données d'ouvrages et renommons les colonnes pour qu'elles soient plus facile à traiter. Le fichier concerné est _ouvrages-surverses.csv_. Pour plus d'informations sur ce jeu de données: http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse.

In [267]:
ouvrages = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_ENV", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(ouvrages, Symbol.(colnames));

┌ Warning: `names!(df::AbstractDataFrame, vals::Vector{Symbol}; makeunique::Bool=false)` is deprecated, use `rename!(df, vals, makeunique=makeunique)` instead.
│   caller = top-level scope at In[267]:3
└ @ Core In[267]:3


<a id="2.4.1."></a>
#### 2.4.1. Données discrètes

Tout d'abord, procédons à une analyse des données discrètes des ouvrages. Les données discrètes de ce jeu de données sont:

- N_ENV: Identifiant de l'ouvrage Environnement Canada.
- ID_SOMA: Identifiant de l'ouvrage selon SOMAEU (Gouvernement du Québec).
- ID_OUVRAGE: Identifiant de l'ouvrage selon la ville.
- NOM: Nom de l'ouvrage.
- SOMA_SEC: Nom du secteur SOMAEU où se trouve l'ouvrage.
- REGION: Nom de la région (arrondissement ou municipalité) où se trouve l'ouvrage.

Analysons brièvement chacune de ces variables afin de déterminer si elles pourraient jouer le rôle de variable explicative dans notre situation.

*N_ENV, ID_SOMA, ID_OUVRAGE, NOM:*

In [ ]:
random_first(ouvrages, 10, [:N_ENV, :ID_SOMA, :ID_OUVRAGE, :NOM])

Comme on peut le voir, chaque ouvrage possède une valeur unique pour ces colonnes. Celà fait du sens, compte tenu qu'il s'agit d'un identifiant unique à l'ouvrage, exprimé sous plusieurs normes.
<br>

En raison du faible pouvoir prédictif d'une telle propriété, dûe à l'unicité de chaque valeur, et de la haute corrélation entre les variables, nous avons décidé de ne pas inclure ces variables dans notre modèle. Nous conservons évidemment l'identifiant de l'ouvrage (_ID_OUVRAGE_) qui va permettre d'identifier chaque ouvrage et de joindre plusieurs jeux de données ensemble, mais les autres ne semblent pas posséder de pouvoir prédictif significatif.
<br>

Il est à noter que le numéro contenu dans _N_ENV_ et _ID_SOMA_, ainsi que les 3 lettres dans _ID_SOMA_ pourraient tous deux permettrent de grouper certains ouvrages ensembles ou de les localiser dans une carte, mais d'autres variables explicatives abordées à la section [2.4.1. Données discrètes](#2.4.1.) et [2.4.2. Données de Trop-Plein](#2.4.2.) s'occupent déjà de cette localisation. 

_SOMA_SEC, REGION_

Observons maintenant les effets de ces deux variables discrètes. Tout d'abord, le secteur selon la norme SOMAEU. Visualisons la carte de Montréal selon les positions de Trop-Pleins, identifiant le secteur SOMAEU par couleur.

In [ ]:
plot(ouvrages,x=:TP_LNG, y=:TP_LAT, Geom.point, color= :SOMA_SEC)

Comme on peut le voir, la majorité des ouvrages sont groupés sous le secteur _OMAEU de Montreal (Station Jean-R.-Marcotte_). Ce problème d'imbalancement de classes pourrait s'avérer problématique. Effectuons maintenant la même tâche pour la séparation de secteurs selon leur arrondissement ou municipalité.

In [ ]:
plot(ouvrages,x=:TP_LNG, y=:TP_LAT, Geom.point, color= :REGION)

Les secteurs sont beaucoups mieux balancés. Cette variable explicative permet de grouper ensemble un certain nombres d'ouvrages, ce qui pourrait s'avérer utile pour la prédiction.

On remarque cependant que certaines régions ne contiennent que très peu d'ouvrages. Il pourrait être intéressant de grouper ensemble des régions contigues afin d'obtenir un nombre d'ouvrage par région plus balancé encore.

<a id="2.4.2."></a>
#### 2.4.2. Données de trop-plein

Passons maintenant à l'analyse des données de trop-plein. Nous avons ici 5 variables explicatives pour le trop-plein.

- TP_X: Coordonnée en X selon la norme NAD83 MTM8.
- TP_Y: Coordonnée en Y selon la norme NAD83 MTM8
- TP_Z: Coordonnée en Z selon la norme NAD83 MTM8.
- TP_LNG: Longitude selon la norme WGS84.
- TP_LAT: Latitude selon la norme WGS84.

Pour ce qui est de l'interprétation des trops-pleins, bien que nous ayons eu certaines difficultés à correctement interpréter le sens de ce terme en comparaison avec les émissaires, comme discuté dans la section [5.3. Difficultés rencontrées](#5.3.), des recherches plus poussées en ligne nous ont guidé vers le sens du trop-plein dans ce contexte: Il s'agit ici de l'ouvrage qui permet aux eaux non dirigées vers la station d'épuration d'être évacuées vers le milieu récepteur naturel<sup>[[1]](#refs)</sup>. 

Les couples de coordonnées [X, Y] et [LNG, LAT] semblent redondantes, testons cette hypothèse.

*TP_X, TP_Y, TP_LNG, TP_LAT:*

In [ ]:
plt_x_y = plot(ouvrages, x=:TP_X, y=:TP_Y, Geom.point);
plt_lng_lat = plot(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point);
hstack(plt_x_y, plt_lng_lat)

Comme on peut le voir sur les graphiques, ces données sont en effet plutôt similaires et mappent toutes les deux le contour de l'Ile de Montréal, où sont situés les Trop-Pleins. Nous avons choisi de ne garder pour l'instant que les latitudes et longitudes, ce format étant consistent avec les données présentes dans un autre jeu de données.

*TP_Z:*

Observons maintenant la répartition des hauteurs de trop-plein.

In [ ]:
plot(ouvrages, x=:TP_Z, Geom.histogram(bincount=50), Guide.xlabel("Hauteur de trop-plein"),Guide.ylabel("Fréquence"))

Une analyse sur les données de hauteur nous informent que cette information est manquante pour une partie des ouvrages. 

In [ ]:
describe(ouvrages[!, :TP_Z])

Comme on peut le voir, environ 10% des données sont manquantes. Une solution à ce problème est proposée dans la section [3. Traitement des données](#3.).

<a id="2.4.3."></a>
#### 2.4.3. Données d'emissaire

Concluons l'analyse des données d'ouvrages avec les variables concernant l'émissaire. Selon la description des données accompagnant le jeu de données sur le site de Montréal, le positionnement géographique des ouvrages de débordement est associé aux émissaires dans cet ensemble de données<sup>[[2]](#refs)</sup>. 

Encore une fois, ces coordonnées sont présentes selon les deux normes, avec le couple [X,Y] ou le couple [LNG, LAT]. Nous choisissons le second afin de rester consistent avec nos autres choix jusqu'à présent.

Ces données contiennent des valeurs manquantes, comme l'indique la description des colonnes.

In [ ]:
describe(ouvrages[!, :EMI_LNG])

In [ ]:
describe(ouvrages[!, :EMI_LAT])

Cela correspond aux indications présentes sur le site web. Le traitement des données manquantes sera abordé dans la section [3. Traitement des données](#3.). Pour l'instant, nous allons ignorer les données manquantes afin d'observer le graphique des émissaires.

In [ ]:
emi_lng_no_missing = collect(skipmissing(ouvrages[!, :EMI_LNG]));
emi_lat_no_missing = collect(skipmissing(ouvrages[!, :EMI_LAT]));
plot(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point)

Comme on peut le voir, la forme ressemble fortement à celle obtenue avec les données de trop-pleins. Stackons les deux pour voir la différence et comparons verticalement le résultat lorsque chacun est par dessus l'autre.

In [ ]:
tp_first_emi = plot(layer(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point, order=1, Theme(default_color=colorant="orange")),
     layer(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point, order=2, Theme(default_color=colorant="blue")),
     Guide.title("Émissaire par dessus"), Guide.xlabel(nothing),Guide.ylabel(nothing));

emi_first_tp = plot(layer(ouvrages, x=:TP_LNG, y=:TP_LAT, Geom.point, order=2, Theme(default_color=colorant="orange")),
     layer(x=emi_lng_no_missing, y=emi_lat_no_missing, Geom.point, order=1, Theme(default_color=colorant="blue")),
     Guide.title("Trop-plein par dessus"), Guide.xlabel(nothing),Guide.ylabel(nothing));

hstack(tp_first_emi, emi_first_tp)

Comme on peut le voir, ces coordonnées sont assez semblables. Utiliser les deux dans notre modèle pourrait donc entrainer des problèmes de corrélation. Nous avons, pour la majorité du temps de travail, utilisé les coordonnées de trop-pleins afin d'effectuer nos prédictions, n'utilisant donc pas les données d'émissaires dans notre modèle. Ce choix provient du fait que les données d'émissaires sont parfois manquantes (10% du temps).
<br>

Une autre approche utilisée récemment nous permet de ne pas avoir à choisir entre les deux et de ne pas utiliser de données géographiques. Cette approche sera discutée dans la section [2.7. Isolation des ouvrages d'intérêts](#2.7).

<a id="2.5."></a>
### 2.5. Analyse des données de précipitations

<a id="2.5.1."></a>
#### 2.5.1. Données de précipitation

Passons à l'analyse des données de précipitations à certaines stations pluviométrique. Nous devons effectuer des fonctions d'aggrégation sur ce tableau avant de pouvoir l'intégré dans notre jeu de donnée de nos modèles. En effet, ce tableau contient les données de précipitation par heure pour chacune des cinqs stations météorologiques près de montréal. Cependant, notre jeu de données des features est seulement par jour. Nous allons donc applique 3 fonctions d'aggrégation différente, soit : Somme, Maximum et Maximum3h. 

Ce jeu de données a été préparé par notre professeur à partir des données météos d'environnement Canada<sup>[[3]](#refs)</sup> et contient la date sous le format yyyy-mm-jj, l'heure, ainsi que la quantité de pluie au dixième de millimètre pour chacune des cinqs stations pluviométriques suivante :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Les dates contenues dans ce dataset comprennent les années 2013 à 2019. Par contre, nous devons entraîner notre modèle sur les années avant 2019, et prédire sur toute l'année 2019 (de mai à octobre). Il est donc important de noter que cette table contient des données de l'ensemble d'entraînement et de test.

Commençons par importer les données de précipitations et renommer la colonne St-Hubert afin qu'elle soit plus facile à traiter.

In [17]:
precipitations = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitations, Symbol("St-Hubert")=>:StHubert);

Nous ne sommes intéressés que par les mois de Mai à Octobre, inclusivement. Analysons les valeurs associés à ce lapse de temps.

In [18]:
precipitations = filter(row -> month(row.date) > 4, precipitations);
precipitations = filter(row -> month(row.date) < 11, precipitations);
random_first(precipitations, 5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2018-05-01,21,0,0,0,0,0
2,2014-06-04,23,missing,0,0,0,0
3,2013-07-12,23,0,0,0,0,missing
4,2016-06-08,18,0,0,0,0,0
5,2019-10-28,3,2,missing,missing,0,0


Comme on peut l'observer ci-dessous, plusieurs de ces données sont manquantes.

In [19]:
describe(precipitations)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,Type
1,date,,2013-05-01,,2019-10-31,1288,,Date
2,heure,11.5,0,11.5,23,,,Int64
3,McTavish,1.42821,0,0.0,2082,,1585,"Union{Missing, Int64}"
4,Bellevue,1.16081,0,0.0,295,,4010,"Union{Missing, Int64}"
5,Assomption,1.40768,0,0.0,326,,1789,"Union{Missing, Int64}"
6,Trudeau,1.21048,0,0.0,366,,182,"Union{Missing, Int64}"
7,StHubert,1.22368,0,0.0,307,,5206,"Union{Missing, Int64}"


In [ ]:
describe(precipitations[!, :McTavish])

 La stratégie de résolution des données manquantes est discutée dans la section [3. Traitement des données](#3.). Pour l'instant, gardons seulement les données complêtes à des fins d'observations.

In [ ]:
dropmissing!(precipitations);
size(precipitations, 1)

<a id="2.5.2."></a>
#### 2.5.2. Somme des précipitations journalières

Commençons par obtenir la somme des précipitations dans une journée, par station pluviométrique.

In [ ]:
pcp_sum = by(precipitations, :date,  
            McTavish = :McTavish=>sum, 
            Bellevue = :Bellevue=>sum,
            Assomption = :Assomption=>sum, 
            Trudeau = :Trudeau=>sum, 
            StHubert = :StHubert=>sum);

Il y a beaucoup d'éléments sur ce graphique. Une particularité de Gadfly est qu'il nous permet de choisir les séries à afficher. On peut cliquer donc cliquer sur la station d'intérêt afin de filtrer les résultats. On peut remarquer sur ce graphique que les stations ont des tendances similaires, mais pas totalement identiques. Si nous prennons McTavish et Assomption pour l'année 2016, nous pouvons voir que la majorité des pics se superposent, avec quelques exceptions où des pics plus bas apparaissent seulement pour une station.

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Somme des précipitations des stations météo pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Il semblerait donc, jusqu'à présent, que McTavish reçoive plus de précipitations en moyenne que les autres stations. Voyons si cette tendance se maintient pour les autres années.

In [ ]:
df_for_plot = melt(pcp_sum, :date)
plot(df_for_plot, x=:date, y=:value, Geom.point, color=:variable, Guide.title("Somme des précipitations des stations météo de 2013 à 2018"), Guide.xlabel("Date (j)"),  Guide.ylabel("Précipitation (mm 10^-1)"))

La tendance semble se conserver, mais l'année 2019 est très différente, beaucoup d'informations sont manquantes. Analysons séparément l'année 2019.

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2019, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Somme des précipitations des stations météo pour 2019"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Comme on peut le voir, nous ne possédons des données de précipitations que pour la majorité du mois de Mai. En réalité, une analyse de la table de précipitation nous informe que nous possédons les informations couvrant la majorité de l'année pour certaines stations, mais d'autres, comme Bellevue, ne sont couvertes que jusqu'en Mai. Une solution à ce problème sera abordé à la section [3. Traitement des données](#3.).

<a id="2.5.3."></a>
#### 2.5.3. Taux horaire maximal des précipitations journalières

Passons au taux horaire maximal par jour des précipitations.

In [ ]:
pcp_max = by(precipitations, :date,  
            McTavish = :McTavish=>maximum,
            Bellevue = :Bellevue=>maximum, 
            Assomption = :Assomption=>maximum,
            Trudeau = :Trudeau=>maximum,
            StHubert = :StHubert=>maximum);

Visualisation de ces informations pour l'année 2016:

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_max);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Taux horaire maximal des précipitations pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Le graphique semble suggérer une tendance similaire à celle que la somme des précipitations journalières nous procure. Le reste des informations étant assez similaires entre la somme et le taux horaire maximal, nous ne montrerons pas de graphes additionels dans cette sous-section.

<a id="2.5.4."></a>
#### 2.5.4. Taux sur trois heures maximal des précipitations journalières

Finalement, voyons le taux sur trois heures maximal des précipitations.

In [ ]:
pcp_max3 = by(precipitations, :date,
                McTavish = :McTavish=>maximum3,
                Bellevue = :Bellevue=>maximum3,
                Assomption = :Assomption=>maximum3,
                Trudeau = :Trudeau=>maximum3,
                StHubert = :StHubert=>maximum3);

Visualisation de ces informations pour l'année 2016:

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2016, pcp_max3);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable, Guide.title("Taux maximal sur 3 heures pour 2016"), Guide.xlabel("Date (j)"), Guide.ylabel("Précipitation (mm 10^-1)"))

Encore une fois, nous pouvons voir des positions de pics similaires entre les trois graphiques, avec des hauteurs de pics différentes compte tenu des types de valeurs variables.

En effet, la somme des précipitations pour une journée sera toujours supérieure ou égale au taux maximal sur 3h qui lui même sera toujours supérieur ou égal au taux horaire maximal. Démontrons cette hypothèse avec les données pour McTavish en 2018:

In [ ]:
date_to_pt = 2018;
loc = :McTavish
mct_sum = pcp_sum[:,[1,2]]
rename!(mct_sum, loc => :Sum);
mct_sum = filter(row -> year(row.date) == date_to_pt, mct_sum);

mct_max = pcp_max[:,[1,2]]
rename!(mct_max,loc => :Max);
mct_max = filter(row -> year(row.date) == date_to_pt, mct_max);

mct_max3 = pcp_max3[:,[1,2]]
rename!(mct_max3,loc => :Max3);
mct_max3 = filter(row -> year(row.date) == date_to_pt, mct_max3);

df_for_plot = join(mct_sum, mct_max3, on = :date);
df_for_plot = join(df_for_plot, mct_max, on = :date);
df_for_plot = melt(df_for_plot, :date)


plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

<a id="2.6."></a>
### 2.6. Analyse des données de surverses

<a id="2.6.1"></a>
#### 2.6.1. Chargement des surverses pour analyse

Enfin, nous chargeons les données du fichier <i>Surverses.csv</i>. Ce jeu de donnée provient des données ouvertes de Montréal : http://donnees.ville.montreal.qc.ca/dataset/debordement. Le fichier ne correspond pas tout à fait aux données en ligne. Il y a eu des transformations (jointures) apportées par notre professeur. Le fichier est structuré de la sorte:
<br>

**NO_Ouvrage** : (String) L'idendifiant unique de chacun des ouvrages <br>
**Date** : (Date) La date sous le format yyyy-mm-jj de l'observation <br>
**Surverse** : (Bool) Si l'ouvrage a surversé à cette date <br>
**Raison** : (String) Abréviation de la raison de la surverse. Les options sont variables, Ex:
- "U" : Urgence;
- "Inconnue" : La raison est inconnue;
- "TS" : Déversement par temps sec;
- "P" : Débordement du à la pluie; 
- "N" : Rejet à la rivière des Prairies;
- "S" : Rejet au fleuve St-Laurent;

Au final, nous ne garderons que les trois premières colonnes. La colonne NO_Ouvrage sera renommé afin d'être consistente avec l'identifiant de l'ouvrage des autres jeux de données. De plus, nous ne garderons que les surverses dûes à la pluie, à un débordement par temps sec ou à une raison inconnue que nous considérerons comme étant dûe à la pluie.

Chargeons les données de surverses:

In [86]:
surverses = CSV.read("data/surverses.csv", missingstring="-99999");

Nous ne gardons que les mois de Mai à Octobre inclusivement.

In [87]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

Conservons les surverses dûes à la pluie, au temps sec ou de raison inconnue.

In [88]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

Les données de surverses manquantes sont retirées: On ne peut rien analyser avec celles ci.

In [89]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);

<a id="2.6.2."></a>
#### 2.6.2. Lien entre ouvrages et surverses

Existe t'il des ouvrages qui sont plus susceptibles à des surverses que d'autres? C'est ce que nous allons chercher à déterminer dans cette sous-section. Commençons par joindre ces deux jeux de données ensemble.

In [ ]:
ouvrages_surverses = join(ouvrages, surverses, on =:ID_OUVRAGE);
select!(ouvrages_surverses, [:ID_OUVRAGE, :SURVERSE, :TP_LNG, :TP_LAT, :TP_Z]);

Voyons si la position des ouvrages influence le nombre de surverses reçues.

In [ ]:
n_surverses_by_ouvrage = by(ouvrages_surverses, :ID_OUVRAGE, 
    SURVERSE = :SURVERSE => sum,
    TP_LNG = :TP_LNG => mean, 
    TP_LAT = :TP_LAT => mean,
    TP_Z = :TP_Z => mean);
plot(n_surverses_by_ouvrage, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:SURVERSE)

À première vue, il seble que la moitié droite de l'Île de Montréal soit plus touchée par des surverses que la moitié gauche. On observe aussi le plus de surverses au croisement milieu de l'Île. La position semble donc influencer la quantité de surverses qu'un ouvrage subit durant l'année. On se rend compte en revanche que c'est principalement la longitude qui joue en rôle dans la quantité de surverses plutôt que la longitude.

Ceci n'est évidemment qu'une observation qui semble indiquer une corrélation, mais ne constitue pas une preuve de corrélation. Nous tiendrons néanmoins compte de ce résultat dans notre traitement de données puis dans notre sélection de modèles.

Quand est il de la hauteur du trop plein? Observons son influence sur la quantité de surverses.

In [ ]:
dropmissing!(n_surverses_by_ouvrage, disallowmissing=true);
plot(n_surverses_by_ouvrage, x=:TP_Z, y=:SURVERSE, Geom.hair)

Contrairement à notre intuition, la hauteur ne semble pas jouer un rôle si important sur la quantité de surverses d'un trop-plein.

<a id="2.6.3."></a>
#### 2.6.3. Lien entre précipitations et surverses

Trouvons maintenant le lien entre le taux de surverses et la quantité de précipitation reçus par les ouvrages. Le facteur commun direct entre les deux est la date, groupons donc ces données par dates.

On commence par renommer la colonne :date des précipitations afin de pouvoir effectuer la jointure.

In [ ]:
rename!(pcp_sum, :date => :DATE)
rename!(pcp_max, :date => :DATE)
rename!(pcp_max3, :date => :DATE)

On effectue la jointure et on groupe par date afin de visualiser les données.

In [ ]:
pcp_sum_surverses = join(pcp_sum, surverses, on=:DATE)
select!(pcp_sum_surverses, Not([:ID_OUVRAGE]))
n_surverses_by_date = by(pcp_sum_surverses, :DATE, 
                            McTavish = :McTavish => first,
                            Bellevue = :Bellevue => first,
                            Assomption = :Assomption => first,
                            Trudeau = :Trudeau => first,
                            StHubert = :StHubert => first,
                            SURVERSE = :SURVERSE => sum);

Observons l'effet de la quantité de pluie reçus dans une journée à McTavish sur le taux de précipitation.

In [ ]:
plot(n_surverses_by_date, x=:McTavish, y=:SURVERSE, Geom.point)

Comme nous pouvons le remarquer, il existe une relation bien définie entre la quantité de pluie reçue et le nombre de surverses dans une journée.

Voyons si ce phénomène est présent pour les autres stations pluviométriques.

In [ ]:
bellevue_plot = plot(n_surverses_by_date, x=:Bellevue, y=:SURVERSE, Geom.point);
assomption_plot = plot(n_surverses_by_date, x=:Assomption, y=:SURVERSE, Geom.point);
trudeau_plot = plot(n_surverses_by_date, x=:Trudeau, y=:SURVERSE, Geom.point);
st_hubert_plot = plot(n_surverses_by_date, x=:StHubert, y=:SURVERSE, Geom.point);
gridstack([bellevue_plot assomption_plot; trudeau_plot st_hubert_plot])

Bien que la relation ne soit pas aussi clairement définie partout, il existe toutefois un lien entre la quantité reçue par une station et le nombre de surverses sur l'Île de Montréal. Nous tiendrons compte de cette information lors du traitement des données et de l'entrainement des modèles.

Ce même évènement devrait se produire pour pcp_max et pcp_max3, étant donné qu'ils sont très corrélés avec pcp_sum. Assurons nous que ce soit le cas avec pcp_max.

In [ ]:
pcp_max_surverses = join(pcp_max, surverses, on=:DATE)
select!(pcp_max_surverses, Not([:ID_OUVRAGE]))
n_surverses_by_date = by(pcp_max_surverses, :DATE, 
                            McTavish = :McTavish => first,
                            Bellevue = :Bellevue => first,
                            Assomption = :Assomption => first,
                            Trudeau = :Trudeau => first,
                            StHubert = :StHubert => first,
                            SURVERSE = :SURVERSE => sum);

Reprenons la station McTavish pour analyser le phénomène.

In [ ]:
plot(n_surverses_by_date, x=:McTavish, y=:SURVERSE, Geom.point)

Comme on pouvait l'imaginer, le graphe est très similaire à celui du pcp_sum. 
Afin de garder le rapport concis, nous ne démontrerons pas ce résultat pour les autres stations et pour pcp_max3 ici.

<a id="2.7."></a>
### 2.7. Isolation des ouvrages d'intérêts

<a id="2.7.1."></a>
#### 2.7.1. Motivation

Une approche qui a été prise plus tard dans le processus de développement de notre modèle a été d'isoler nos ouvrages. En effet, nous savons déjà quels ouvrages seront utilisés pour la prédiction finale. Il conviendrait alors de ne se concentrer que sur ces ouvrages dans la création de nos modèles. 

Ce sujet a été abordé par l'équipe dès la première journée du projet, et rapidement disciminé en faveur d'un entrainement sur toutes les données. Il a finalement été revu et mis en oeuvre dans les derniers jours de rafinement du modèle afin d'améliorer encore plus les résultats. 

Voici les arguments en faveur de cette approche:
- Diminution du nombre de données d'entrées, donc permet un entrainement de modèle plus rapide.
- Beaucoup de données d'entrées sont invariables aux ouvrages, ils n'influencent pas les chances de surverses au niveau de l'ouvrage en tel quel. 
- Permet d'avoir un modèle spécialisé par ouvrage, qui sera probablement plus précis qu'un modèle général.

Voici les arguments contre cette approche:
- Diminution du nombre de données d'entrées, donc moins d'informations pour rendre notre modèle plus robuste.
- Ne serait pas capable de prédire les surverses pour un ouvrage autre que ceux entrainés.

À la lumière de ces arguments, nous avons finalement choisi d'isoler nos ouvrages et d'entrainer un modèle par ouvrage. En effet, nous savons quels ouvrages vont être utilisés et ne cherchons pas à prédire les surverses pour d'autres ouvrages dans le cadre de ce devoir. 

De plus, nous éliminons ainsi un bon nombre de variables explicatives qui ne joue pas un rôle direct dans la prédiction du taux de surverses par ouvrage. Par exemple, la position d'un ouvrage ne change pas et donc cette variable n'explique plus la fluctuation des surverses pour cet ouvrage. Nous pouvons alors nous concentrer sur les seuls éléments qui changent: les données de précipitations.

Ce faisant, nous éliminons donc une grande partie de l'effort que nous avons mis dans la conception d'un modèle général, y compris le traitement de certaines qui ne seront alors plus utilisés. Nous expliquerons ce que nous ferons à ce sujet dans la section [3. Traitement des données](#3.).

<a id="2.7.2."></a>
#### 2.7.2. Chargement des données traitées

Dans le but de pouvoir analyser les données de chaque ouvrage d'intérêt séparément, ils sont passés par un processus rafinement qui sera couvert dans la section [3. Traitement des données](#3.). Afin de ne pas polluer la partie analyse du rapport, nous allons directement charger les données qui ont été générées plus tard.

Prenons l'ouvrage 3260-01D pour commencer.

In [5]:
ouvrage_3260 = CSV.read("data/parsed/ouvrage_3260.csv");

Ce jeu de données contient les données de précipitation pour les deux stations les plus proches de l'ouvrage, dont la somme et le maximum sur 3h des précipitations, pour chaque jour. 

De plus, il contient aussi l'information de surverse pour chaque date enregistrée.

In [ ]:
random_first(ouvrage_3260, 5)

Ici, FS correspond à _First Station_ et SS correspond à _Second Station_. Les colonnes _dist_ correspond à la distance entre l'ouvrage et la station correspondante. Pour ce qui des données de _sum_ , _max_ et _max3_ , elles sont standardisées avec les données complètes et traitées de toutes les stations.

<a id="2.7.3."></a>
#### 2.7.3. Station la plus proche

Voyons maintenant s'il y'a une relation entre les données de précipitation de la station la plus proche et le taux de surverses.

In [ ]:
sum_plot = plot(ouvrage_3260, x=:SURVERSE, y=:FS_sum, Geom.boxplot);
max_plot = plot(ouvrage_3260, x=:SURVERSE, y=:FS_max, Geom.boxplot);
max3_plot = plot(ouvrage_3260, x=:SURVERSE, y=:FS_max3, Geom.boxplot);
hstack([sum_plot, max_plot, max3_plot])

Il semblerait d'après ce graphe en boites que la somme des précipitations est le indice de surverses. Observons le lien entre la somme et le taux maximum pour cet ouvrage.

In [ ]:
plot(ouvrage_3260, x=:FS_sum, y=:FS_max, Geom.point, color=:SURVERSE)

Comme on pouvait l'imaginer, ces deux variables sont très corrélées. Certains modèles ne sont pas affectés par une telle correlation mais d'autres, tel que la régression logistique, peut avoir des difficultés avec une telle corrélation. 

<a id="2.7.4."></a>
#### 2.7.4. Deuxième station la plus proche [TODO: Add image to illustrate]

La question à se poser maintenant est la suivante: Existe t'il un lien entre la deuxième station la plus proche et le taux de précipitations?

En effet, nous avons tout d'abord assumé que seule la station la plus proche d'un ouvrage influençait son taux de précipitation. Cette idée est fausse, car si un ouvrage se trouve à l'intersection de deux stations, et que des précipitations ne touchent qu'une des deux stations, ou en touche une plus que l'autre, l'ouvrage pourrait être affecté.

En réalité, un ouvrage pourrait être à l'intersection de plus que deux stations, mais les chances sont très faibles et l'impact sur la prédiction, limité. Nous avons donc décidé de nous limiter aux deux stations les plus proches.

Maintenant, il faut tenir compte du fait que ces deux stations n'ont pas le même effet sur l'ouvrage. La station la plus proche est, dans la majorité des cas, celle qui va expliquer le mieux surverses d'un ouvrage. Ainsi, nous diminuons l'impact de la deuxième station par un facteur qui dépends de distance avec l'ouvrage. Plus de détails sur cette modification à la section [3. Traitement des données](#3.).

Observons l'impact de la deuxième station par rapport à la première station pour les précipitations de cet ouvrage.

In [ ]:
plot(ouvrage_3260, x=:FS_sum, y=:SS_sum, Geom.point, color=:SURVERSE)

Ce graphique montre une majorité de surverses à l'intersection de ces deux stations, mais aussi lorsque la deuxième station reçoit plus de précipitations que la station la plus proche. Ceci valide notre hypothèse concernant l'influance des autres stations sur le taux de surverses.

On peut voir la distance entre l'ouvrage et les stations ainsi:

In [ ]:
ouvrage_3260[1, [:FS_dist, :SS_dist]]

La deuxième station est donc $ \frac{0.210769}{0.145981} = 1.44$, soit 44% plus loin de l'ouvrage que la station la plus proche.

Encore une fois, afin de ne pas rendre cette section trop longue, nous ne présenterons pas les autres ouvrages ici. Les détails spécifiques à chaque ouvrage seront présentés à la section [3. Traitement des données](#3.).

<a id="2.8."></a>
### 2.8. Retour sur l'analyse

En conclusion, cette analyse nous a permis de repérer les informations pertinentes à notre projet et d'isoler les variables d'intérêt. Elle nous a aussi permis de détecter les possibles causes d'erreurs et a joué un rôle important pour la tâche subséquente, soit le traitement de ces données. Elle a enfin permis de renforcer ou discréditer certaines hypothèses _a priori_ que nous avions sur les données afin de les incorporer ou les retirer de nos modèles. 

Cette étape a ainsi constitué la partie la plus longue et fastidieuse de notre projet, mais a grandement servi à l'amélioration de nos modèles.

<a id="3."></a>
## 3. Traitement des données [TODO: UPDATE TABLE DES MATIERES]

<a id="3.1."></a>
### 3.1. Motivation

Cette section aborde une étape cruciale lors de la mise en oeuvre de notre solution. C'est en effet ici que les données sont modifiées après l'analyse de l'étape précédente afin de servir de base à nos modèles.

Le choix adéquat des variables explicatives ainsi que leur rafinement jouent un rôle tout aussi important que le choix du modèle, et va grandement influencer les performances de celui ci. C'est pourquoi nous avons consacré une grande partie de notre temps à traiter et peaufiner les données pour ce projet.

De plus, certains modèles ne fonctionnent bien qu'en présence de données traitées d'une certaine manière. Un tel modèle serait, par exemple, la régression logistique, qui fonctionne de manière bien plus efficace lorsque toutes les variables explicatives sont à la même échelle. La régression logisitique, ainsi que la classification bayésienne naive, marchent aussi mieux lorsque les données ne sont pas trop corrélées entre elles. C'est pourquoi il est important de s'assurer ici de bien choisir et structurer nos données avant de les offrir à nos modèles.

<a id="3.2."></a>
### 3.2. Processus de traitement

Nous allons présenter dans cette section les étapes et choix faits pour la préparation et traitements des données d'entrées à nos modèles. Le processus de traitement de données a été le suivant:

1. Sélection des données à utiliser
2. Modification des données afin d'entrainer les modèles
3. Entrainement des modèles et validation
4. Détection des problèmes liés aux données
5. Correction et amélioration des données

Ces étapes ont été répétés en boucle tout au long du projet, en raison du nombre important de variations des modèles sélectionnés.

<a id="3.3."></a>
### 3.3. Préchargement des données

# TODO: Remove maybe?

<a id="3.4."></a>
### 3.4. Considérations et étapes retenues [TODO: UPDATE TABLE DES MATIERES]

Dans cette sous-section, nous présenterons chacune des considérations de traitements de données ainsi que leurs motivations. Pour les considérations qui ont été retenues, nous irons en détails sur leur implémentation pour le modèle final. 

En effet, bien que chacune de ces considérations ait été implémentée et testée pour entrainer des modèles et effectuer des prédictions, seulement une partie a été utile et donc gardée pour notre modèle final. 

Afin de ne pas rendre ce rapport plus long que nécessaire, tout en gardant un historique concis du travail effectué, nous expliquerons ici les options de traitements de données.

<a id="3.4.1."></a>
#### 3.4.1. Normalisation de chacune des colonnes entre 0 et 1

La première considération a été de normaliser chacune des colonnes entre 0 et 1. Cette modification a été implémentée afin d'uniformiser les données pour la régression logistique car ce modèle est très sensible aux différences d'ordre de grandeur entre les données. Les autres modèles sont beaucoups moins, voir pas du tout, affecté par ces différences.

En effet, nos premiers modèles utilisants les informations de positions ( _TP_LNG, TP_LAT_ ), de hauteur ( _TP_Z_ ) et de précipitations ( _PCP_SUM, PCP_MAX, PCP_MAX3_ ) avaient affaire à des ordres de grandeurs très différents.

Par exemple, les données de géolocalisations pour la longitude se trouvaient toutes autour de 45.5 $\pm$ 0.5 tandis que la lattitude tournait autour de -73.5 $\pm$ 0.5. Les données de précipitations allaient de 0 à 2500, avant traitement, et cette échelle variait d'une colonne de précipitation à une autre. Les hauteurs de trop-plein quant à elles variaient de 9 à 35 mètres en général.

L'approche utilisée pour normaliser ces données a été de diviser chaque rangée de chaque colonne par la valeur maximale de cette colonne.

Nous avons cependant remplacé par la suite cette approche en faveur de la standardisation des colonnes, qui sera expliquée dans la partie [3.4.3. Standardisation de chacune des colonnes](#3.4.3.). Ainsi, nous ne détaillerons pas les détails d'implémentations de la normalisation des colonnes dans cette sous-section.

<a id="3.4.2."></a>
#### 3.4.2. Traitement des données de hauteur manquantes

Nos premiers modèles utilisant l'information de hauteur des trop-pleins, il était important de ne pas avoir de valeurs manquantes qui pourraient fausser nos résultats. L'approche initiale a été d'assigner la valeur de 0 aux données manquantes, mais a rapidement été remplacé par la moyenne des hauteurs de trop-plein. 

En effet, il ne fait pas de sens pour un trop-plein d'avoir une hauteur de 0, sachant que la plupart des données se trouvaient autour de 10 à 30m et que les valeurs minimales avant traitement étaient de 9. La moyenne de 19.32 permettait de ne pas trop diverger de cette zone lors de nos prédictions.

En revanche, le modèle retenu étant celui des ouvrages isolés, les données de trop-plein qui sont invariables à chaque ouvrage n'étaient plus pertinentes. Ainsi, ce traitement n'a pas été retenu pour notre modèle final et son implémentation ne sera pas détaillé ici.

<a id="3.4.3."></a>
#### 3.4.3. Standardisation de chacune des colonnes [TODO: EXPLIQUER RAISON]

La standardisation des colonnes a été utilisée à la place de la normalisation de celles ci en raison de **TODO**. 

Cette standardisation s'est avérée très utile lors de la réalisation de nos premières prédictions qui utilisaient des modèles entrainés sur l'ensemble des données. En revanche, nos prédictions subséquentes ont été réalisées avec des modèles spécialisés par ouvrage qui utilisaient des données relativement uniformes en ordre de grandeur. 

L'intérêt de la standardisation s'est donc vu grandement atténué. Il n'en reste qu'aucun des modèles utilisés ne souffre d'une telle standardisation et que celle ci pourrait s'avérer à nouveau utile si nous nous étions retrouvés à ajouter des données d'échelles différentes à nos modèles. De plus, le coût de standardisation était faible et celui ci n'est ajouté que lorsque des changements au traitement de données sont effectués. Nous avons donc décidé de conserver cette étape. 

Pour chacune des colonnes contenant des valeurs continues dans nos données de prédiction, nous effectuons la modification suivante:

In [15]:
function standardize_col(col)
    mean_col = mean(col);
    std_col = std(col);
    
    res = (col .- mean_col) ./ std_col;
    
    return res;
end

standardize_col (generic function with 1 method)

<a id="3.4.4."></a>
#### 3.4.4. Filtrage des mois de précipitations et surverses [TODO: UPDATE NUMERING FROM HERE]

Comme abordé puis implémenté dans la section [2. Analyse exploratoire](#2.), nous ne conservons que les données de précipitations et de surverses entre Mai et Octobre. 

En effet, la très grande majorité des précipitations se produisent durant cette période et c'est aussi à ce moment que la majorité des surverses sont causés par la pluie. En dehors de cette période, la neige constituerait le facteur le plus probable pour une surverse, dans le cas où les données sont manquantes. De plus, nos prédictions s'effectuent sur des données situés entre Mai et Octobre, nous n'avons pas à tenir compte des tendances des autres périodes, qui représentent une incertitude considérable.

Rechargeons les données de précipitations pour leur traitement

In [82]:
precipitations = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitations, Symbol("St-Hubert")=>:StHubert);
precipitations = filter(row -> month(row.date) > 4, precipitations);
precipitations = filter(row -> month(row.date) < 11, precipitations);

<a id="3.4.5."></a>
#### 3.4.5. Traitement des données de précipitations abérantes

La description des données de précipitations semble indiquer des valeurs maximales assez élevées par rapport à la moyenne.

In [83]:
describe(precipitations)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,Type
1,date,,2013-05-01,,2019-10-31,1288,,Date
2,heure,11.5,0,11.5,23,,,Int64
3,McTavish,1.42821,0,0.0,2082,,1585,"Union{Missing, Int64}"
4,Bellevue,1.16081,0,0.0,295,,4010,"Union{Missing, Int64}"
5,Assomption,1.40768,0,0.0,326,,1789,"Union{Missing, Int64}"
6,Trudeau,1.21048,0,0.0,366,,182,"Union{Missing, Int64}"
7,StHubert,1.22368,0,0.0,307,,5206,"Union{Missing, Int64}"


Cette fonction permet de traiter les valeurs manquantes pour la somme des colonnes.

In [29]:
function sum_with_missing(col)
    somme = 0;
    for i=1:size(col, 1)
        if !isequal(col[i], missing)
            somme += col[i];
        end
    end
    return somme;
end

sum_with_missing (generic function with 1 method)

Le graphe suivant permet d'observer ce phénomène, avec la station McTavish comme example

In [ ]:
precipitations_per_day = by(precipitations, :date,
                            McTavish= :McTavish => sum_with_missing,
                            Bellevue= :Bellevue => sum_with_missing,
                            Assomption= :Assomption => sum_with_missing,
                            Trudeau= :Trudeau => sum_with_missing,
                            StHubert= :StHubert => sum_with_missing);
precipitations_to_plot = melt(precipitations_per_day, :date)
plot(precipitations_to_plot, x=:date, y=:value, color=:variable)

Comme on peut le voir, certaines données sont bien au dessus de la moyenne. Prenons les valeurs abérantes au dessus de 2000 pour McTavish.

In [106]:
idx_outliers = precipitations[!, :McTavish] .> 2000;
idx_outliers[isequal.(idx_outliers, missing)] .= false;
idx_outliers = convert(Array{Bool, 1}, idx_outliers);

date_outlier = precipitations[idx_outliers, :date];
precipitations[idx_outliers, :]

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-10-10,13,2082,0,0,0,missing


Une seule date contient la valeur au dessus de 2000, quand est il des autres stations à cette date? Les autres stations ne rapportent pas de précipitations à cette date là. Y'a t'il eu des surverses dans nos ouvrages d'intérets, le 10 Octobre 2013?

In [107]:
important_ouvrages = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"];
filter(row -> row.DATE == date_outlier[1] && row.ID_OUVRAGE ∈ important_ouvrages, surverses)

,ID_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,3260-01D,2013-10-10,0
2,4240-01D,2013-10-10,0
3,4350-01D,2013-10-10,0
4,4380-01D,2013-10-10,0


Visiblement, il y'a pas eu de surverses à cette date là, remplaçons donc cette donnée par 0 afin qu'elle n'induise pas en erreur notre modèle.

In [113]:
precipitations[idx_outliers, :McTavish] .= 0;
precipitations[idx_outliers, :StHubert] .= 0;
precipitations[idx_outliers, :]

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-10-10,13,0,0,0,0,0


La plus grosse donnée abérante a été résolue, regardons le graphique de précipitations à nouveau

In [ ]:
precipitations_per_day = by(precipitations, :date,
                            McTavish= :McTavish => sum_with_missing,
                            Bellevue= :Bellevue => sum_with_missing,
                            Assomption= :Assomption => sum_with_missing,
                            Trudeau= :Trudeau => sum_with_missing,
                            StHubert= :StHubert => sum_with_missing);
precipitations_to_plot = melt(precipitations_per_day, :date)
plot(precipitations_to_plot, x=:date, y=:value, color=:variable)

Voyons combien de données au dessus de 200 on obtient.

In [ ]:
idx_outliers = precipitations[!, :McTavish] .>= 200;
idx_outliers[isequal.(idx_outliers, missing)] .= false;
idx_outliers = convert(Array{Bool, 1}, idx_outliers);

date_outlier = precipitations[idx_outliers, :date];
precipitations[idx_outliers, :]

On observe 7 valeurs au dessus de 200 pour 29,327 entrées. La partie qui suit est assez arbitraire: Nous regardons chacune des 7 dates au dessus de 240 et vérifions s'il y a surverse, si ce n'est pas le cas, nous enlevons les surverses abérantes.

In [ ]:
precipitations[precipitations.date .== Date(2013, 7, 17), :]

Ici nous pouvons voir que le 240 apparait de nul part, et constitue une valeur extrèmes de précipitations. Il n'a pas plu de la journée et d'un coup, 240 mm d'eau s'est déversé sur la station. Bien que cela reste possible, s'il n'y a pas eu de surverses à cette journée là, il est peu probable que cette donnée soit utile à notre modèle.

In [ ]:
filter(row -> row.DATE == Date(2013, 7, 17) && row.ID_OUVRAGE ∈ important_ouvrages, surverses)

Comme il y a eu surverse, il est fort probable que le haut taux de pluie à cette journée ait eu lieu et ait causé la surverse, nous ne toucherons donc pas à cette donnée.

Répétons le processus pour chacun des autres ouvrages, et éliminons les données abérantes qui ne causent pas de surverses. Les étapes sont les mêmes que pour précédemment, nous éviterons donc de les redétailler.

In [251]:
precipitations[precipitations.date .== Date(2017, 7, 20), :Bellevue] .= 0;
precipitations[precipitations.date .== Date(2017, 7, 20), :Trudeau] .= 0; # VOIR SI GARDER LUI

precipitations[precipitations.date .== Date(2013, 6, 24), :Assomption] .= 0;
precipitations[precipitations.date .== Date(2014, 8, 5), :Assomption] .= 0;
precipitations[precipitations.date .== Date(2015, 6, 10), :Assomption] .= 0;
precipitations[precipitations.date .== Date(2018, 7, 26), :Trudeau] .= 0;

Ainsi, nous avons éliminé toutes les précipitations hors du commun ne menant pas à des surverses. Celles ci sont probablement dûes à des erreurs de mesure, ou autres, mais dans tous les cas faussent nos résultats. Cette approche reste tout de fois très arbitraire et sera discuté dans la section [5.3. Améliorations possibles](#5.3.).

<a id="3.4.4."></a>
#### 3.4.4. Traitement des données de précipitations manquantes

Le jeu de données _precipitations.csv_ possédait aussi un nombre considérable de données manquantes. Il fallait alors trouver une solution pour travailler avec ces données en limitant la perte d'informations.

Certaines journées étaient entièrement dépourvues de données de précipitations pour une station. L'approche utilisée à donc été d'assigner un taux de précipitation nul à ces journées. Une amélioration possible sera abordé à la section [5.3. Améliorations possibles](#5.3.). 

Pour les journées possédant des certaines valeurs manquantes mais pas toutes, nous prennons la moyenne du reste des valeurs de la journée pour la station concernée, et l'appliquons à chacune des valeurs manquantes. La plupart du temps, ces valeurs sont de 0 ou proche, la moyenne générale de chaque station n'est donc pas tant affectée.

La fonction suivante permet de trouver la moyenne de précipitation d'une journée contenant des valeurs manquantes:

In [252]:
function mean_wo_missing(col)
    mean = 0;

    for i=1:size(col, 1)
        if !isequal(col[i, 1], missing)
            mean += col[i, 1]
        end
    end
    mean = mean ÷ size(col, 1)
    return mean;
end

mean_wo_missing (generic function with 1 method)

Appliquons cette fonction à nos données de précipitations

In [253]:
precipitation_by_day = by(precipitations, :date,  
                            McTavish = :McTavish=>mean_wo_missing, 
                            Bellevue = :Bellevue=>mean_wo_missing, 
                            Assomption = :Assomption=>mean_wo_missing,
                            Trudeau = :Trudeau=>mean_wo_missing,
                            StHubert = :StHubert=>mean_wo_missing)

for i=1:size(precipitations,1)
    if isequal(precipitations[i, :McTavish], missing)
        precipitations[i,:McTavish] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:McTavish][1]
    end
    if isequal(precipitations[i, :Bellevue], missing)
        precipitations[i,:Bellevue] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Bellevue][1]
    end
    if isequal(precipitations[i, :Assomption], missing)
        precipitations[i,:Assomption] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Assomption][1]
    end
    if isequal(precipitations[i, :Trudeau], missing)
        precipitations[i,:Trudeau] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Trudeau][1]
    end
    if isequal(precipitations[i, :StHubert], missing)
        precipitations[i,:StHubert] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:StHubert][1]
    end
end

Observons maintenant les données de précipitations, après traitement de données abérantes et manquantes

In [ ]:
precipitations_per_day = by(precipitations, :date,
                            McTavish= :McTavish => mean,
                            Bellevue= :Bellevue => mean,
                            Assomption= :Assomption => mean,
                            Trudeau= :Trudeau => mean,
                            StHubert= :StHubert => mean);
precipitations_to_plot = melt(precipitations_per_day, :date)
plot(precipitations_to_plot, x=:date, y=:value, color=:variable)

Ce jeu de données semble maintenant près à être utilisé pour la suite du processus de traitement de données.

<a id="3.4.5."></a>
#### 3.4.5. Traitement des données de surverses manquantes

En ce qui concerne les surverses, nous éliminons toutes les entrées qui ne contiennent pas des informations de surverses, ainsi que toutes celles en dehors de la période de Mai à Octobre.

En effet, nous ne sommes intéressés que par les données qui vont servir à effectuer une prédiction et la variable de surverse constitue l'élément à prédire par nos modèles dans un contexte d'apprentissage supervisé. Il n'est donc pas possible d'entrainer, dans ce même contexte, un modèle sans cette variable.

De plus, comme expliqué à la section [2. Analyse exploratoire](#2.), nous ne considèrerons que les surverses dûes à la précipitations dans ce projet. Voici le code qui permet de nettoyer nos données de surverses, déjà employé à la section 2.

In [256]:
surverses = CSV.read("data/surverses.csv", missingstring="-99999");
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);
surverses[!,:RAISON] = coalesce.(surverses[:,:RAISON],"Inconnue");

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
surverses = filter(row -> row.ID_OUVRAGE ∈ important_ouvrages, surverses);
dropmissing!(surverses);

<a id="3.4.4."></a>
#### 3.4.4. Ajout de la somme et du taux maximal de précipitations

Les précipitations étant séparés en heures, plutôt qu'en journées comme le reste des données, nous devons la convertir en ce dernier afin de pouvoir utiliser ces données.

Plusieurs stratégies de conversion s'offrent à nous, pour chaque jour, à chaque station, nous pouvons considérer:
- La somme des précipitations de la journée
- Le taux maximal de précipitations de la journée
- La moyenne de précipitations de la journée
- Le mode de précipitations de la journée

Nous avons décidé d'utiliser les deux premiers, car notre interprétation d'une surverse nous dit que celle ci est généralement causé par un débit inatendu d'eau sur une courte période plutôt qu'un flux continu. Ce choix est cependant arbitraire et constitue une amélioration possible à notre projet, comme discuté à la section [5.3. Améliorations possibles](#5.3.).

Somme des précipitations quotidiennes par station

In [261]:
pcp_sum = by(precipitations, :date,  
            McTavish = :McTavish=>sum, 
            Bellevue = :Bellevue=>sum,
            Assomption = :Assomption=>sum, 
            Trudeau = :Trudeau=>sum, 
            StHubert = :StHubert=>sum);

Taux horaire maximal de précipitations dans une journée, par station

In [260]:
pcp_max = by(precipitations, :date,  
            McTavish = :McTavish=>maximum,
            Bellevue = :Bellevue=>maximum, 
            Assomption = :Assomption=>maximum,
            Trudeau = :Trudeau=>maximum,
            StHubert = :StHubert=>maximum);

Cette partie a été discutée à la section [2. Analyse exploiratoire](#2.), et nous n'afficherons pas de graphes ici afin de ne pas dupliquer l'information.

<a id="3.4.4."></a>
#### 3.4.4. Ajout du taux maximal de précipitation sur 3 heures

Partant de l'hypothèse qu'une surverse est causée par un débit inatendu d'eau dans un trop-plein, sur une durée limitée, il est naturel que le taux maximal sur 1h soit considéré. 

Une autre possibilité serait le taux sur 3h maximal de précipitations, offrant une plus grosse marge. Encore une fois assez arbitraire comme choix, nous avons estimé que celà pourrait s'ajouter comme variable explicative à notre modèle afin d'aider à la prédiction.

Il reste que cette donnée est très corrélée avec la somme et le maximum de précipitations, il pourrait donc causer problèmes pour la régression logistique ou la classification bayésienne naive.

In [262]:
pcp_max3 = by(precipitations, :date,
                McTavish = :McTavish=>maximum3,
                Bellevue = :Bellevue=>maximum3,
                Assomption = :Assomption=>maximum3,
                Trudeau = :Trudeau=>maximum3,
                StHubert = :StHubert=>maximum3);

La fonction maximum3 a été définie à la section [2.1. Analyse exploratoire] lors de la visualisation de cette donnée.

<a id="3.4.4."></a>
#### 3.4.4. Conservation des données pour la station la plus proche

Par la suite, nous avons supposé que la station la plus proche aurait le plus d'influence sur les surverses d'un ouvrage. Ainsi, nous avons récupéré les localisations géographiques des stations pluviométriques à l'aide d'une recherche internet et bati un DataFrame à partir de celles ci.

In [263]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish", 45.504742, -73.579167]);
push!(station_df, ["Bellevue", 45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau", 45.467778, -73.741667]);
push!(station_df, ["StHubert", 45.5175, -73.416944]);

station_df

,STATION,LAT,LNG
,String,Float64,Float64
1,McTavish,45.5047,-73.5792
2,Bellevue,45.4272,-73.9292
3,Assomption,45.8094,-73.4347
4,Trudeau,45.4678,-73.7417
5,StHubert,45.5175,-73.4169


Puis, nous avons comparé les longitudes et latitudes de chaque station avec l'ouvrage en cours, afin de déterminer le plus proche. La fonction suivante a été utilisé pour cela.

In [264]:
function findDistance(oLat, oLng, sLat, sLng)
    lat = (sLat - oLat) ^ 2;
    lng = (sLng - oLng) ^ 2;

    return sqrt(lat + lng);
end

findDistance (generic function with 1 method)

Nous avons ainsi déterminé la station la plus proche de l'ouvrage en cours, et ajouté à cet ouvrage les données de précipitations de la station (somme, maximum, maximum3). Les détails de l'implémentation ne sont pas ajoutés ici car ils seront couverts dans une autre sous-section.

En revanche, nous nous sommes rendus compte que la station la plus proche n'était pas le seul facteur qui déterminait le taux de surverse d'un ouvrage. En raison de la nature de la pluie et de sa zone d'effet, un ouvrage peut être touché par la pluie sans que la station la plus proche de celui ci ne le soit. Il peut aussi être plus touché. Observer un autre ouvrage aiderait à obtenir cette information.

<a id="3.4.4."></a>
#### 3.4.4. Conservation des données pour toutes les stations

À la lumière de la conclusion obtenue à la sous-section précédente, nous avons par la suite décidé de garder les informations de chaque station (somme, maximum, maximum3), afin d'effectuer nos prédictions par ouvrage.

Nos modèles n'étaient en revanche pas tout à fait en mesure de correctement déterminer quel station influençait le plus le taux de surverses d'un ouvrage donné, lorsque les informations de précipitations de plusieurs stations se ressemblaient. 

<a id="3.4.4."></a>
#### 3.4.4. Conservation des données pour les deux stations les plus proches

Afin de résoudre les problèmes encontrés dans les deux sous-sections précédentes, nous avons décidé de garder en mémoire pour chaque couple d'ouvrage / date, les informations des deux ouvrages les plus proches. En effet, ce choix, bien qu'arbitraire encore une fois, est un juste milieu entre nos solutions. Il est aussi plus probable qu'un ouvrage se trouve à l'intersection de deux stations et soit touché par une zone de précipitations qu'il ne le soit de trois ou plus et que la précipitation ne touche qu'une seule de ces stations.

Voici l'implémentation de cette section. Tout d'abord, ajoute les colonnes de précipitations à nos surverses.

In [265]:
train_data = surverses;

train_data[!, :FS_dist] = zeros(size(train_data, 1));
train_data[!, :SS_dist] = zeros(size(train_data, 1));
train_data[!, :FS_sum] = zeros(size(train_data, 1));
train_data[!, :FS_max] = zeros(size(train_data, 1));
train_data[!, :FS_max3] = zeros(size(train_data, 1));
train_data[!, :SS_sum] = zeros(size(train_data, 1));
train_data[!, :SS_max] = zeros(size(train_data, 1));
train_data[!, :SS_max3] = zeros(size(train_data, 1));

Par la suite, on remplit ces colonnes avec les données des deux stations les plus proches.

In [268]:
for i=1:size(train_data, 1)
    curr_ouvrage = train_data[i, 1];
    ouvrage_data = filter(row -> row.ID_OUVRAGE == curr_ouvrage, ouvrages);
    
    closest_station = nothing;
    closest_distance = 9999;
    
    second_closest_station = nothing;
    second_closest_distance = 9999;
    
    # Pour chaque station
    for j=1:5
       current_station = station_df[j, :STATION];
       dist = findDistance(ouvrage_data[1, :TP_LAT], ouvrage_data[1, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);
       
        if dist < closest_distance
            second_closest_distance = closest_distance;
            second_closest_station = closest_station;
            closest_distance = dist;
            closest_station = current_station;
        elseif dist < second_closest_distance
            second_closest_distance = dist;
            second_closest_station = current_station;
        end
    end
    
    train_data[i, :FS_dist] = closest_distance;
    train_data[i, :SS_dist] = second_closest_distance;
    
    # Add data for first station
    sum_p = pcp_sum[∈([train_data[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    train_data[i, :FS_sum] = sum_p[1];
    max_p = pcp_max[∈([train_data[i, :DATE]]).(pcp_max.date), Symbol(closest_station)];
    train_data[i, :FS_max] = max_p[1];
    max3_p = pcp_max3[∈([train_data[i, :DATE]]).(pcp_max3.date), Symbol(closest_station)];
    train_data[i, :FS_max3] = max3_p[1];
    
    # Add data for second station
    s_sum_p = pcp_sum[∈([train_data[i, :DATE]]).(pcp_sum.date), Symbol(second_closest_station)];
    train_data[i, :SS_sum] = s_sum_p[1];
    s_max_p = pcp_max[∈([train_data[i, :DATE]]).(pcp_max.date), Symbol(second_closest_station)];
    train_data[i, :SS_max] = s_max_p[1];
    s_max3_p = pcp_max3[∈([train_data[i, :DATE]]).(pcp_max3.date), Symbol(second_closest_station)];
    train_data[i, :SS_max3] = s_max3_p[1];
end

Nous obtenons ainsi les données de précipitations des deux stations les plus proches, par ouvrage.

In [272]:
random_first(train_data, 5)

,ID_OUVRAGE,DATE,SURVERSE,FS_dist,SS_dist,FS_sum,FS_max,FS_max3
,String,Date,Int64,Float64,Float64,Float64,Float64,Float64
1,4240-01D,2017-08-24,0,0.149987,0.168295,0.0,0.0,0.0
2,4240-01D,2014-06-15,0,0.149987,0.168295,0.0,0.0,0.0
3,4240-01D,2013-09-16,0,0.149987,0.168295,0.0,0.0,0.0
4,3350-07D,2017-05-29,1,0.0927373,0.12027,64.0,26.0,53.0
5,3350-07D,2018-06-26,0,0.0927373,0.12027,0.0,0.0,0.0


Les colonnes _dist_ correspondent à la distance entre l'ouvrage et la station.

<a id="3.4.4."></a>
#### 3.4.4. Mise à l'échelle des données de la seconde station la plus proche

Finalement, une autre considération a été de mettre à l'échelle la contribution de la deuxième station la plus proche d'un ouvrage par rapport à la première. 

Plusieurs approches ont été utilisées avant d'arriver à l'approche qui nous parraissait la plus approprié: Nous avons utilisé une fonction d'adoucicement afin de ne pas trop pénaliser la seconde station. 

Voici notre raisonnement: 
- Si une station est deux fois plus loin de l'ouvrage qu'une autre, sa contribution devrait tourner autour de 80% de la première station, plutot que juste 50%. 
- Si une station est environ aussi loin de l'ouvrage qu'une autre, sa contribution devrait frôler les 100%.
- Si une station est beaucoup plus loin, sa contribution devrait diminuer en conséquence.

Nous avons ainsi généré les distances des deux stations les plus proches pour les 5 ouvrages d'intérets, et nous avons manuellement généré une fonction qui nous semble appropriée. Encore une fois, comme cette fonction a été générée manuellement selon nos observations et intuitions, elle constitue une potentielle faiblesse de notre modèle à améliorer, comme discutée à la section [5.3. Améliorations possibles](#5.3.).

La fonction en question calcule le ratio de distance de la deuxième station par rapport à la première, puis trouve la racine carrée de ce ratio et le passe en argument à un logarithm naturel, ce résultat est ensuite soustrait à un afin d'obtenir la contribution de la seconde station.

Contribution = $ 1 - \ln(\sqrt(\frac{seconde closest distance}{closest distance})) $

Voici l'implémentation de la fonction en question.

In [274]:
function contribution_second(closest_distance, second_closest_distance)
    ratio = second_closest_distance / closest_distance;
    logratio = log(sqrt(ratio));
    return 1 - logratio; 
end

contribution_second (generic function with 1 method)

Cette fonction est finalement appliquée aux données de précipitations de la deuxième station la plus proche avant d'être ajoutée aux données de l'ouvrage.

<a id="3.4.6."></a>
#### 3.4.6. OneHot des dates pour les mois et jours

Finalement, un autre traitement général des données a été de considérer les jours et les mois de l'année comme variable explicative de précipitation.

Nous avons donc encodé les mois entre 5 et 10 et les jours de 1 à 31 et passé cette information à notre modèle.

Ces données ont aussi dûes être standardisées afin de ne pas causer de problèmes d'échelle pour la régression logistique.

Par la suite, nous avons retiré cette variable explicative de notre modèle en raison de son faible pouvoir prédictif par rapport aux autres données. Nous ne détaillerons donc pas ici l'implémentation de ces variables explicatives.

<a id="3.5."></a>
### 3.5. Isolation des ouvrages d'intérêts

Comme vu à la section [2.7. Isolation des ouvrages d'intérêts](#2.7.), nous avons décidé de séparer notre jeu de données final en 5 jeux de données, soit un par ouvrage d'intérêt. **TODO: CONTINUER**

<a id="3.5.1."></a>
#### 3.5.1. Sous-échantillonage de l'ensemble des colonnes d'entrainement

<a id="3.5.2."></a>
#### 3.5.2. Sur-échantillonage de l'ensemble des colonnes d'entrainement

<a id="4."></a>
## 4. Sélection de modèles

<a id="4.1."></a>
### 4.1. Motivation

Maintenant que les données sont prêtes à être utilisées, il est temps de passer à l'étape de sélection de modèles. En effet, le choix du ou des modèles est crucial dans le domaine de la prédiction car, selon les données reçues, certains modèles vont mener à des meilleures prédictions. Il est donc primordial de bien déterminer le type de modèle qui serait adapté à notre situation afin d'obtenir les meilleures performances et résultats possibles.

<a id="4.2."></a>
### 4.2. Choix des modèles

Les modèles que nous avons considéré pour cet exercice sont:

- Les arbres de décision
- La forêt aléatoire
- La machine à vecteur de support (SVM)
- La régression logistique
- La classification bayésienne naive
- L'ensemble de modèles

Chacun de ces modèles sera présenté dans une sous-section de la section [4. Sélection de modèles](#4.), où les avantages et désavantages du modèle seront présentés. Pour les modèles aillant été retenus, nous allons détailler les notions théoriques sur lesquels reposent ces modèles et comment nous les avons appliqué. Pour les modèles non retenus, nous expliqueront pourquoi ils n'ont pas été retenu en faveur des autres.

<a id="4.3."></a>
### 4.3. Arbres de décision et forêt aléatoire [TODO: METTRE A JOUR TABLE DES MATIERES]

<a id="4.3.1."></a>
#### 4.3.1. Théorie des arbres de décision

Le modèle d'arbres de décision est un algorithme d'apprentissage machine très puissant capable d'effectuer des tâches de classification, mais aussi de régression. Dans notre cas, c'est la partie classification qui nous intéresse ici car nous cherchons à déterminer si une surverse a eu lieu ou pas, selon certaines variables explicatives.

Les avantages des arbres de décision sont:
- A
- B
- C

Les désavantages des arbres de décision sont:
- A
- B
- C

Les arbres de décision agissent sur chaque variable en **TODO: CONTINUE**

<a id="4.3.2."></a>
#### 4.3.2. Théorie des forêts aléatoires

<a id="4.3.3."></a>
#### 4.3.3. Définition des fonctions utilitaires

<a id="4.3.4."></a>
#### 4.3.4. Application à l'ensemble de données

<a id="4.3.5."></a>
#### 4.3.5. Application aux ouvrages isolés

<a id="4.4."></a>
### 4.4. Régression logistique

<a id="4.5."></a>
### 4.5. Machine à vecteurs de support

<a id="4.6."></a>
### 4.6. Classification bayésienne naive

<a id="4.7."></a>
### 4.7. Ensemble de modèles

<a id="5."></a>
## 5. Retour et conclusion

<a id="5.3."></a>
### 5.3. Difficultés rencontrées [TODO: UPDATE TABLE DES MATIÈRES]

Lors de cet exercice, nous avons rencontré un certain nombre de difficultés qui nous en ralentit dans la réalisation du projet. Nous allons discuter ici de ces difficultés.

<a id="5.3.1."></a>
#### 5.3.1. Interprétation des données

Une des plus grande difficultés de ce projet fut de correctement interpréter les données. En effet, une erreur de notre part fut de confondre la signification d'émissaire et de trop-plein. Ne sachant pas initialement ce que ces termes représentaient, une recherche Google Image nous a mené à penser que le trop-plein était une sorte de conteneur dans lequel l'eau s'accumulait et l'émissaire comme étant le tuyeau de déversement vers le fleuve. 
<br>

Cela nous a ainsi mené à effectuer des suppositions qui s'avéraient erronées et ont faussé nos résultats pendant une bonne partie de l'exercice.

<a id="5.3.2."></a>
#### 5.3.2. Données manquantes de précipitations

<a id="5.3.3."></a>
#### 5.3.3. Remplacement des données abérantes de précipitations

<a id="5.3.4."></a>
#### 5.3.4. Données des journées alentours

<a id="5.3.5."></a>
#### 5.3.5. Explorer plus de procédés d'aggrégation pour les précipitations

Ex: Moyenne ou mode

<a id="refs"></a>
## 6. Références

## TODO: Référence style APA ou IEEE
+ [1]: http://collections.banq.qc.ca/ark:/52327/bs44911

+ [2]: http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

+ [3]: https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019
